<p align="center">
    <img src="https://github.com/jessepisel/energy_analytics/blob/master/EA_logo.jpg?raw=true" width="220" height="240" />

</p>

# Neural Networks in Numpy
## Freshman Research Initiative Energy Analytics CS 309

#### Jesse Pisel, Assistant Professor of Practice, University of Texas at Austin
**[Twitter](http://twitter.com/geologyjesse)** | **[GitHub](https://github.com/jessepisel)** | **[GoogleScholar](https://scholar.google.com/citations?user=Z4JzYgIAAAAJ&hl=en&oi=ao)** | **[LinkedIn](https://www.linkedin.com/in/jesse-pisel-70519430/)**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

We are going to do our normal import of the data and get the labels encoded like we have for the past few weeks

In [2]:
data = pd.read_csv(r'well_data.csv') #read it in
data.tail()

,DEPT,AHT10,AHT20,AHT30,AHT60,AHT90,AHTCO60,AHTCO90,DPHZ,DSOZ,...,ITT,NPOR,PEFZ,RSOZ,RXOZ,SDEV,SP,SPHI,RHOZ,TOP
5466,1914.0,1.6167,3.0335,7.5475,8.5244,9.1691,117.3103,109.0619,-0.4129,0.5048,...,0.2649,0.4847,10.0000,0.0348,2.9599,1.0538,1.6250,0.7009,3.3313,MATANUSKA
5467,1913.5,1.6164,3.0324,7.5492,8.5195,9.1830,117.3782,108.8963,-0.6763,0.3208,...,0.2650,0.4760,10.0000,0.0000,1.7452,1.0770,10.9375,0.6161,3.7659,MATANUSKA
5468,1913.0,1.6163,3.0317,7.5488,8.5243,9.1852,117.3116,108.8711,-0.9772,0.2371,...,0.2651,0.4754,10.0000,0.0000,0.3407,1.0509,43.8125,0.5991,4.2624,MATANUSKA
5469,1912.5,1.6162,3.0311,7.5493,8.5248,9.1936,117.3051,108.7711,-1.1748,0.2120,...,0.2652,0.4853,10.0000,0.0000,0.2168,0.8236,79.5000,0.6521,4.5884,MATANUSKA
5470,1912.0,1.6161,3.0305,7.5496,8.5289,9.1974,117.2483,108.7263,-1.1654,0.2080,...,0.2652,0.4471,9.9845,0.0000,0.1797,0.7958,108.5000,0.6699,4.5729,MATANUSKA


In [3]:
from sklearn import preprocessing #for label encoding
#label encode our formation data
le = preprocessing.LabelEncoder()
top_names = data.TOP
le.fit(data.TOP)
tops = le.transform(data.TOP)
data.drop('TOP', axis=1, inplace=True)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.values, tops, test_size=0.2, random_state=86)

#### We have our data organized and split, now let's define some functions for our network

First we want to figure out what kind of activation function we want to use. Let's go with a sigmoid function of the form:

$$\sigma{(x)}=\frac{1}{1+e^{-x}} $$


In [5]:
def activation(x):
    return 1 / (1 + np.exp(-x))

Now we want to define a function that calculates our prediction using our weights, bias, and features of the form:

$$\hat{y} = \sigma(w_1 x_1 + w_2 x_2 + b)$$

We can express the RHS as:
$$
\sigma(\begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}+b)
$$

Remember we want to take the dot product of the features and weights, add the bias, then run the entire thing through the activation function. Let's do a forward pass through this layer with random weights for one feature

In [6]:
np.random.seed(86)
features = X_train[0] #features for one sample
r_weights = np.random.rand(features.shape[0])
r_bias = np.random.rand(1)
y_hat = activation(np.matmul(features, r_weights)+r_bias)
y_hat

array([1.])

This is the output for a single neuron. Neurons come into their own when you stack the individual perceptrons into layers and stacks of layers into networks. In the case of a two layer network, the output of one layer becomes the input for the next layer. The network can be expressed as:

$$
\hat{y} =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

where $W_1$ are the weights for the first layer and $W_2$ are the weights for the second layer.
Let's calculate the forward pass through a two layer network for all the neurons and one sample

In [7]:
np.random.seed(86)
n_input = X_train.shape[1]
n_hidden = 2
n_output = 1
features = X_train[0]
target = y_train[0]
# Weights for inputs to hidden layer
W1 = np.random.randn(n_input, n_hidden)
# Weights for hidden layer to output layer
W2 = np.random.randn(n_hidden, n_output)

# and bias terms for hidden and output layers
B1 = np.random.randn(1, n_hidden)
B2 = np.random.randn(1, n_output)

Next we want to multiply our features with our weights, add the bias, and run it through the activation. After that, take the ouput and multiply it with the second weights matrix, and add the second bias. This is our early prediction that needs to be scaled with some kind of activation function. We will do that next

In [8]:
h = activation(np.matmul(features, W1) + B1)
y_hat = np.matmul(h, W2) + B2
y_hat

C:\Users\jrp4932\AppData\Local\Continuum\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


array([[-0.66494327]])

Next, let's start to build a new network. Since we have 27 features, let's make a network that takes in 27 input units, has 54 hidden units, and has two output units (one for each of our classes). We are going to use our sigmoid activation for the hidden layer of the network. 

In [9]:
n_input = X_train.shape[1] #27 features
n_hidden = 54
n_output = 2
features = X_train[0]
target = y_train[0]
# Weights for inputs to hidden layer
W1 = np.random.randn(n_input, n_hidden)
# Weights for hidden layer to output layer
W2 = np.random.randn(n_hidden, n_output)

# and bias terms for hidden and output layers
B1 = np.random.randn(1, n_hidden)
B2 = np.random.randn(1, n_output)

In [10]:
h = activation(np.matmul(features, W1)+B1)
y_hat = np.matmul(h, W2)+B2
y_hat

C:\Users\jrp4932\AppData\Local\Continuum\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


array([[6.19994504, 2.53728256]])

This output from the network looks somewhat useful, but it would be more useful if we could calculate a probability distribution from it, so that we could select the class with the highest probability. To do this, we are going to use a softmax function 
$$
\Large \sigma(x_i) = \cfrac{e^{x_i}}{\sum_k^K{e^{x_k}}}
$$

What this does is scale each input $x_i$ between 0 and 1 and normalizes the values to give a probability distribution equal to one.

In [11]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x))

If we run our predictions through the softmax function, we get the probability for each class. Here we see that the first class has a 99% probability and the second class has a 1% probability

In [12]:
softmax(y_hat)

array([[0.97497807, 0.02502193]])

As you can see, as we add layers into the network it will get much more challenging to track what is happening at each layer. So to deal with this we are going to use `PyTorch`